In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [131]:
import matplotlib
import matplotlib.pyplot as plt

In [4]:
import time

In [5]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

In [6]:
import logging
logging.basicConfig(filename='p3ht_et.log',level=logging.DEBUG)

Now let's create project and resource to keep track of our work

In [7]:
p3ht_et = streamm.Project('P3HT_ET')

In [9]:
p3ht_et.import_json()
for calc_i in p3ht_et.calculations.iteritems():
    print calc_i

And a resource object to keep track of where our files are 

In [11]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [12]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [13]:
res_local.make_dir()

In [14]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [15]:
peregrine = streamm.Resource('peregrine')

In [16]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [17]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [18]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [19]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [20]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [21]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():

    if( particle_i.symbol == 'C' ):
        particle_i.label = "C%d"%(c_cnt)
        particle_i.resname = "SCP2"
        particle_i.residue = 1

        c_cnt +=1 
    if( particle_i.symbol == 'S' ):
        particle_i.resname = "ThS"
        particle_i.residue = 2

    if( particle_i.symbol == 'H' ):
        particle_i.label = "H%d"%(h_cnt)
        particle_i.resname = "HA"
        particle_i.residue = 3

        h_cnt +=1 


Set the force-field type and guess some reasonable charges 

In [22]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
    if( particle_i.symbol == 'C' ):
        particle_i.paramkey = 'CA'
        particle_i.charge = -0.025
    if( particle_i.symbol == 'S' ):
        particle_i.paramkey = 'S'
        particle_i.charge = -0.3
    if( particle_i.symbol == 'H' ):
        particle_i.paramkey = 'HA'
        particle_i.charge = 0.1

Check molecule is neutral 

In [23]:
total_charge = 0.0
for pkey_i, particle_i  in bbTh.particles.iteritems():
    total_charge += particle_i.charge
print total_charge

-2.77555756156e-17


Optimize structure with NWChem

But let's put it in a function this time

In [27]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    calc_n =  len(project_i.calculations)     
    nwchem_i = streamm.NWChem('nw_opt_{}_calc_{}'.format(bb_i.tag,calc_n))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.export_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [28]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene_calc_0
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper/nw_opt_thiop

In [29]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene_calc_0']

Check status unit finished

In [30]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [31]:
print nwchem_i.meta['status']

finished


In [32]:
nwchem_i.analysis()

Print energies 

In [34]:
print nwchem_i.properties['energy'],nwchem_i.unit_conf['energy']

-551.190758104 Ha


Check that the positions of the structure have been optimized 

In [35]:
print bbTh.positions

[[ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
 [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
 [  3.47615240e-01  -5.79042180e-01  -8.10000000e-04]
 [ -6.58844760e-01   3.61010820e-01   0.00000000e+00]
 [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
 [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
 [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
 [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
 [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]]


In [36]:
bbTh.unit_conf['length']

u'ang'

In [37]:
print nwchem_i.strucC.positions

[[ -1.52896184e+00  -1.96220294e+00  -8.01740000e-04]
 [ -1.87314220e-01  -1.91410534e+00  -1.04619000e-03]
 [  3.42508490e-01  -5.70353440e-01  -6.76450000e-04]
 [ -6.05169850e-01   3.80532190e-01  -4.87700000e-05]
 [ -2.25058851e+00  -3.24210270e-01   2.87000000e-05]
 [ -2.16222169e+00  -2.82020399e+00  -1.04323000e-03]
 [  4.31356700e-01  -2.78797334e+00  -1.43211000e-03]
 [  1.39104061e+00  -3.54125130e-01  -9.23550000e-04]
 [ -4.82488530e-01   1.43974965e+00   3.85340000e-04]]


In [39]:
nwchem_i.strucC.unit_conf['length']

u'ang'

Update positions with optimized geometry 

In [40]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.52896184e+00  -1.96220294e+00  -8.01740000e-04]
1 C [ -1.87314220e-01  -1.91410534e+00  -1.04619000e-03]
2 C [ 0.34250849 -0.57035344 -0.00067645]
3 C [ -6.05169850e-01   3.80532190e-01  -4.87700000e-05]
4 S [ -2.25058851e+00  -3.24210270e-01   2.87000000e-05]
5 H [ -2.16222169e+00  -2.82020399e+00  -1.04323000e-03]
6 H [  4.31356700e-01  -2.78797334e+00  -1.43211000e-03]
7 H [  1.39104061e+00  -3.54125130e-01  -9.23550000e-04]
8 H [ -4.82488530e-01   1.43974965e+00   3.85340000e-04]


Store the results in a tar ball in the storage directory 

In [41]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [47]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    calc_n =  len(project_i.calculations)     
    nwchem_esp = streamm.NWChem('nw_esp_{}_calc_{}'.format(bb_i.tag,calc_n))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem_esp.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'SCF'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    nwchem_esp.export_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [65]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

nw_esp_thiophene_calc_1
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52896184      -1.96220294      -0.00080174 \n     C      -0.18731422      -1.91410534      -0.00104619 \n     C       0.34250849      -0.57035344      -0.00067645 \n     C      -0.60516985       0.38053219      -0.00004877 \n     S      -2.25058851      -0.32421027       0.00002870 \n     H      -2.16222169      -2.82020399      -0.00104323 \n     H       0.43135670      -2.78797334      -0.00143211 \n     H       1.39104061      -0.35412513      -0.00092355 \n     H      -0.48248853       1.43974965       0.00038534 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper/nw_esp_thiop

Check status until finished

In [66]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene_calc_1']

In [67]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(1)    

In [68]:
p3ht_et.check()

Calculation nw_esp_thiophene_calc_1 has status finished
Calculation nw_opt_thiophene_calc_0 has status stored


In [69]:
nwchem_i.analysis()

In [70]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge
    total_charge += p.charge
print total_charge

0 C -0.085804
1 C -0.136764
2 C -0.085772
3 C -0.108896
4 S -0.117737
5 H 0.153767
6 H 0.119003
7 H 0.109132
8 H 0.15307
-1.00000000006e-06


Hum a little extra charge can cause problems with our MD simulation so let's round and set to neutral 

In [71]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    p.charge = round(p.charge,4)
    total_charge += p.charge
print total_charge

-2.77555756156e-17


Update the charges of the Buildingblock

In [73]:
bbTh.tag += '_HFesp'

In [74]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.0858
1 C -0.1368
2 C -0.0858
3 C -0.1089
4 S -0.1177
5 H 0.1538
6 H 0.119
7 H 0.1091
8 H 0.1531


Store the results 

In [75]:
nwchem_i.store()

In [76]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [77]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

Store a object of the Buildingblock

In [78]:
os.chdir(res_local.dir['materials']) 
th_json = bbTh.export_json()

Let us optimize the structure with the oplsaa force-field to check the parameters 

In [79]:
os.chdir(res_local.dir['home']) 

In [80]:
from pathlib2 import Path

In [81]:
need_files = ['oplsaa_param.json']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

Need to run forcefields.ipynb


In [82]:
oplsaa = streamm.Parameters('oplsaa')

In [83]:
oplsaa.import_json(read_file=True)

In [84]:
print oplsaa


    Parameters 
      LJ parameters 2 
      Bond parameters 2 
      Angle parameters 2 
      Dihedral parameters 1 
      Improper Dihedral parameters 0 



In [85]:
print oplsaa.unit_conf['energy']

kCalmol


We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [86]:
import streamm.forcefields.particletype as particletype

In [87]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [88]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [89]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [90]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [91]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [92]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [93]:
print CA,HA

 CA epsilon:0.07 sigma:3.55  HA epsilon:0.03 sigma:2.42


In [94]:
S = particletype.Particletype('S')

In [95]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [96]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [97]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [98]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [99]:
import streamm.forcefields.bondtype as bondtype

In [100]:
bt_i = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.080,367.0)
oplsaa.add_bondtype(bt_i)

In [101]:
bt_i = bondtype.Bondtype('CA','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.400,469.0)
oplsaa.add_bondtype(bt_i)

In [102]:
bt_i = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.71,250.0)
oplsaa.add_bondtype(bt_i)

In [103]:
for btk,bt in oplsaa.bondtypes.iteritems():
    print btk,bt

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
1  bond  CT - CT type harmonic 
  harmonic r_0 = 1.530000 K = 268.000000 lammps index 0  gromacs index 0  
2  bond  CA - HA type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
3  bond  CA - CA type harmonic 
  harmonic r_0 = 1.400000 K = 469.000000 lammps index 0  gromacs index 0  
4  bond  S - CA type harmonic 
  harmonic r_0 = 1.710000 K = 250.000000 lammps index 0  gromacs index 0  


In [104]:
import streamm.forcefields.angletype as angletype

In [105]:
bat_i = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,63.0)
oplsaa.add_angletype(bat_i)

In [106]:
bat_i = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [107]:
bat_i = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(92.2,70.0)
oplsaa.add_angletype(bat_i)

In [108]:
bat_i = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [109]:
bat_i = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(111.0,70.0)
oplsaa.add_angletype(bat_i)

In [110]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
2  angle  CA - CA - CA type harmonic 
  harmonic theta_0 = 120.000000 K = 63.000000 lammps index 0  gromacs index 0  
3  angle  CA - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
4  angle  CA - S - CA type harmonic 
  harmonic theta_0 = 92.200000 K = 70.000000 lammps index 0  gromacs index 0  
5  angle  S - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
6  angle  S - CA - CA type harmonic 
  harmonic theta_0 = 111.000000 K = 70.000000 lammps index 0  gromacs index 0  


Set some reasonable dihedral parameters

In [111]:
import streamm.forcefields.dihtype as dihtype

In [112]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [113]:
dih_i = dihtype.Dihtype('X','S','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,2.416710,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [114]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [115]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - S - CA - X type opls 
  k1 = 0.000000 k2 = 2.416710 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 


Let us make an MD simulation of just the monomer to check that our parameters are okay

In [119]:
def lmp_run(project_i,bb_i,param_i,res_i,md_type = 'min'):
    # Create LAMMPS calculation object 
    calc_n =  len(project_i.calculations)     
    lmp_i = streamm.LAMMPS('lmp_{}_{}_calc_{}'.format(md_type,bb_i.tag,calc_n))
    # lmp_i = streamm.LAMMPS('lmp_{}_{}'.format(md_type,bb_i.tag))
    # Set parameter container 
    lmp_i.paramC = param_i
    lmp_i.set_strucC(bb_i)
    # Set force-field parameters 
    lmp_i.set_ffparam()
    # Set resource to local
    lmp_i.set_resource(res_i)
    # Make local directories
    lmp_i.make_dir()
    # Set pbc's to on
    lmp_i.strucC.lat.pbcs = [True,True,True]
    # Change to launch directory
    os.chdir(lmp_i.dir['launch'])
    # Copy over the templates from the template directory 
    lmp_i.cp_file('templates','in',"lammps_{}.in".format(md_type),'templates','launch')
    lmp_i.cp_file('templates','run',"lammps_peregrine.pbs",'templates','launch')
    # Change to scratch
    os.chdir(lmp_i.dir['launch'])
    # Read in template files and store them as strings in the `str` dictionary
    lmp_i.load_str('templates','in')
    lmp_i.load_str('templates','run')
    # Write LAMMPS .data file
    lmp_i.write_data()
    # Replace keys in template string with properties 
    lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))
    # Add the input file to the properties to be written into the run file
    lmp_i.properties['input_in'] = lmp_i.files['input']['in']
    lmp_i.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_i.tag))
    # Save json file in root directory
    os.chdir(lmp_i.dir['home'])
    lmp_i.export_json()
    # Run bash script or submit to cluster
    lmp_i.add_file('output','log',"%s.log"%(lmp_i.tag))
    # Save details in .json files 
    os.chdir(lmp_i.dir['home'])
    lmp_i.export_json()
    #
    os.chdir(lmp_i.dir['launch'])
    lmp_i.push()
    lmp_i.run()
    # Add calculation to project
    project_i.add_calc(lmp_i,deepcopy = True)
    # 
    return project_i     

In [125]:
print p3ht_et.calculations.keys()

['lmp_min_thiophene_HFesp_calc_2', 'nw_esp_thiophene_calc_1', 'nw_opt_thiophene_calc_0']


In [122]:
p3ht_et = lmp_run(p3ht_et,bbTh,oplsaa,peregrine)

In [126]:
lmp_i = p3ht_et.calculations['lmp_min_thiophene_HFesp_calc_2']

In [127]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)    

In [128]:
p3ht_et.check()

Calculation lmp_min_thiophene_HFesp_calc_2 has status finished
Calculation nw_esp_thiophene_calc_1 has status stored
Calculation nw_opt_thiophene_calc_0 has status stored


In [129]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [138]:
lmp_i.store()

Read in data file positions

In [139]:
lmp_i.pull()

Read in data file output and update positions

In [140]:
os.chdir(lmp_i.dir['launch'])

In [141]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [142]:
lmp_i.read_data_pos(datafn)

In [143]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [144]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [145]:
bbTh.tag += '_oplsaa'

In [146]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.56851713e+00  -1.93715379e+00  -7.77923648e-04]
1 C [ -1.72741320e-01  -1.89183487e+00  -1.04556692e-03]
2 C [ 0.33795577 -0.59665857 -0.00068027]
3 C [ -6.51207680e-01   3.89166430e-01  -3.98843176e-05]
4 S [ -2.21178295e+00  -3.39484063e-01   1.34400424e-05]
5 H [ -2.16026366e+00  -2.84213660e+00  -1.04273638e-03]
6 H [  4.46494902e-01  -2.77649802e+00  -1.45175602e-03]
7 H [  1.39431850e+00  -3.72804931e-01  -9.11557221e-04]
8 H [ -4.66096432e-01   1.45451342e+00   3.78252183e-04]


Save the Buildingblock and force-field

In [148]:
os.chdir(res_local.dir['materials']) 
bbTh.write_xyz()
th_json = bbTh.export_json() 
oplsaa_json = oplsaa.export_json()

Okay now that we have a handle on thiophene let's follow the same procedure for hexane

Build hexane

In [149]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [150]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [151]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [152]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

0 C -0.12
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.12
17 H 0.06
18 H 0.06
19 H 0.06


In [153]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

Check that the molecule is neutral 

In [154]:
bbHex.calc_charge()
print bbHex.charge


0.0


Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [156]:
print p3ht_et.calculations.keys()

['lmp_min_thiophene_HFesp_calc_2', 'nw_esp_thiophene_calc_1', 'nw_opt_thiophene_calc_0']


In [155]:
p3ht_et = nw_opt(p3ht_et,bbHex,peregrine)

nw_opt_hexane_calc_3
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.41096900      -0.38164100      -0.00003100 \n     H      -7.31008400       0.24531100      -0.00003800 \n     H      -6.45611700      -1.02879900       0.88463600 \n     H      -6.45611100      -1.02881200      -0.88468900 \n     C      -5.13526800       0.46717500      -0.00003300 \n     H      -5.13548400       1.12878200       0.87797700 \n     H      -5.13547900       1.12877100      -0.87805000 \n     C      -3.85056600      -0.37125800      -0.00002400 \n     H      -3.85112000      -1.03397800       0.87841000 \n     H      -3.85111400      -1.03398700      -0.87845100 \n     C      -2.56745100       0.46960300      -0.00002400 \n     H      -2.56778400       1.13215500       0.87840000 \n     H      -2.56777600       1.13214600      -0.87845500 \n     C      -1.28352700      -0.37023400      -

TypeError: <streamm.calculations.calculation.MDrun object at 0x10e5f5e10> is not JSON serializable

In [ ]:
nwchem_i = p3ht_et.calculations['nw_opt_hexane_calc_3']

In [ ]:
p3ht_et.check()

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

Get the calculation from the project object 

In [ ]:
nwchem_i.analysis()

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Check that the positions of the structure have been optimized 

In [ ]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

In [ ]:
print nwchem_i.strucC.positions

Update positions in Buildingblock object

In [ ]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

Store the results in a tar ball in the storage directory 

In [ ]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [ ]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

Check status unit finished

In [ ]:
p3ht_et.check()

In [ ]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

In [ ]:
nwchem_i.analysis()

In [ ]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

In [ ]:
nwchem_i.strucC.calc_charge()
print nwchem_i.strucC.charge

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Update the charges of the Buildingblock

In [ ]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge

In [ ]:
bbHex.tag += '_HFesp'

Store the results 

In [ ]:
nwchem_i.store()

First we need to identify the bonding within the Buildingblock

In [ ]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [ ]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

Add the need parameters the the oplsaa parameter container

In [ ]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [ ]:
bat_i = angletype.Angletype('CT','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,50.0)
oplsaa.add_angletype(bat_i)

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','HC',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

Run a oplsaa minimization to get the minimized structure

In [ ]:
p3ht_et = lmp_run(p3ht_et,bbHex,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_hexane_HFesp']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
bbHex.tag += '_oplsaa'

In [ ]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
bbHex.write_xyz()
bbHex.dump_pickle() 
oplsaa.dump_pickle()

In [ ]:
print bbHex.tag,bbTh.tag

So let us make some P3HT oligomers 

In [ ]:
os.chdir(res_local.dir['materials']) 

In [ ]:
bbTh.find_rsites()
bbHex.find_rsites()

In [ ]:
print(bbTh.show_rsites())

In [ ]:
print(bbHex.show_rsites())

In [ ]:
import streamm.structures.buildingblock as bb

In [ ]:
ht = bb.attach(bbTh,bbHex,'funccap',0,'rgcap',0,tag='3-hexyl-thiophene')

Update bond angles and dihedrals after Buildingblock join

In [ ]:
ht.bonded_bonds()
ht.bonded_angles()
ht.bonded_dih()

Check that the molecule looks good

In [ ]:
ht.write_xyz()

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [ ]:
ht.calc_charge()
ht.charge

In [ ]:
print(ht.show_rsites())

Add inter thiophene hexane parameters

In [ ]:
bt_i = bondtype.Bondtype('CT','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.51,317.0)
oplsaa.add_bondtype(bt_i)

Bond angle parameters 

In [ ]:
bat_i = angletype.Angletype('CA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,70.0)
oplsaa.add_angletype(bat_i)


bat_i = angletype.Angletype('HA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)



bat_i = angletype.Angletype('CA','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.5,50.0)
oplsaa.add_angletype(bat_i)

bat_i = angletype.Angletype('CA','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(114.0,63.0)
oplsaa.add_angletype(bat_i)

In [ ]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

Note: The inter-ring torsional is not consider as a seperate set of parameters for the simplicity of this example

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

Run a oplsaa minimization to get the minimized structure

In [ ]:
p3ht_et = lmp_run(p3ht_et,ht,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
ht.tag += '_oplsaa'

In [ ]:
for pk,p in ht.particles.iteritems():
    ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,ht.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
ht.write_xyz()
# ht.dump_pickle() 
oplsaa.dump_pickle()

Okay we have the monomer, so let's make a pentamer 

In [ ]:
penta_ht = copy.deepcopy(ht)

In [ ]:
# We could use prepattach to change the tacticity 
# penta_ht = ht.prepattach('termcap',0,dir=-1,yangle=180.0)
# See buildingblock example 

In [ ]:
for n in range(4):
    penta_ht = bb.attach(penta_ht,ht,'termcap',1,'termcap',0,tag='penta_3-hexyl-thiophene')

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [ ]:
penta_ht.calc_charge()
penta_ht.charge

In [ ]:
penta_ht.write_xyz()

Well it's cis, but we can run some high temperature MD to randomize that 

Update bond angles and dihedrals after Buildingblock join

In [ ]:
penta_ht.bonded_bonds()
penta_ht.bonded_angles()
penta_ht.bonded_dih()

In [ ]:
print penta_ht.print_properties()

Run a oplsaa minimization to get the minimized structure

In [ ]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_min_penta_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    # time.sleep(30)

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [ ]:
penta_ht.tag += '_oplsaa'

In [ ]:
for pk,p in penta_ht.particles.iteritems():
    penta_ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,penta_ht.positions[pk]

Save the Buildingblock and force-field

In [ ]:
os.chdir(res_local.dir['materials']) 
penta_ht.write_xyz()
# penta_ht.dump_pickle() 
oplsaa.dump_pickle()

Cool let's run some MD

In [ ]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine,md_type='nvt')

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_nvt_penta_3-hexyl-thiophene_oplsaa']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_3']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

Awesome! We have a randomized pentamer, so let's save that as new Buildingblock

In [ ]:
bbPHTh_1 = copy.deepcopy(lmp_i.strucC)

In [ ]:
print bbPHTh_1

In [ ]:
print bbPHTh_1.n_particles

Now let's replicate the oligomer 50 times to create a low density system

Increase the box size

In [ ]:
pHTh_x = streamm.Buildingblock()

In [ ]:
pHTh_x.lat.matrix = [ 200.,0.,0., 0.,200.,0.,  0.,0.,200.]

In [ ]:
pHTh_x.lat.pbcs = [False,False,False]

In [ ]:
seed = 394572

Randomly place oligomers into the simulation cell

In [ ]:
pHTh_x = streamm.add_struc(pHTh_x,bbPHTh_1,50,seed)

In [ ]:
print pHTh_x.lat.matrix

In [ ]:
pHTh_x.lat.pbcs = [True,True,True]

Check grouping 

In [ ]:
groupset_i = streamm.Groups('mol',pHTh_x)
groupset_i.group_prop('mol','oligomers')

In [ ]:
print len(groupset_i.groups)

In [ ]:
groupset_i.strucC.lat.pbcs

In [ ]:
pHTh_x.tag = 'p3HTx50'

In [ ]:
os.chdir(res_local.dir['materials']) 
pHTh_x.write_xyz()
# lmp_i.strucC.dump_pickle() 

Run a heat cool cycle with NPT to create a solid phase representation of p3HT

In [ ]:
p3ht_et = lmp_run(p3ht_et,pHTh_x,oplsaa,peregrine,md_type = 'equ0')

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_equ0_blank_calc_10']

In [ ]:
print lmp_i.tag

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [ ]:
lmp_i.analysis()

In [ ]:
print lmp_i.properties['run_cnt']

In [ ]:
run_i = lmp_i.properties['run_list'][0]

Plot the time sereies data from the MD runs 

In [133]:
def plot_mdrun(lmp_i):


    fig, ax = plt.subplots(1,sharey=True)
    ax2 = ax.twinx()

    for run_i in lmp_i.properties['run_list']:

        ax.plot(run_i.timeseries['step'],run_i.timeseries['volume'],'b.-')

        ax2.plot(run_i.timeseries['step'],run_i.timeseries['temp'],'k.-')

    ax.set_ylabel('volume', color='b')
    ax2.set_ylabel('temp', color='k')
    ax.set_xlabel('time (fs)', color='k')


    fig.subplots_adjust(hspace=0.0)   
    fig.set_size_inches(8.0, 12.0)

    fig.savefig('{}.pdf'.format(lmp_i.tag),format='pdf')



In [ ]:
plot_mdrun(lmp_i)

Cool the volume is decreasing 

In [ ]:
lmp_i.store()

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_3']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.tag += '_equ0'

In [ ]:
lmp_i.strucC.write_xyz()

In [ ]:
lmp_i.strucC.calc_center_mass()

Create groups out of the molecules

In [ ]:
groupset_i = streamm.Groups('mol',lmp_i.strucC)
groupset_i.group_prop('mol','oligomers')

In [ ]:
print len(groupset_i.groups)

In [ ]:
groupset_i.strucC.lat.pbcs = [False,False,False]

In [ ]:
print groupset_i.strucC.lat.pbcs

In [ ]:
print groupset_i.strucC.lat.matrix

Apply periodic boundries to all the groups, so the molecules are not split across pbc's

In [ ]:
groupset_i.group_pbcs()

In [ ]:
groupset_i.strucC.write_xyz('g_pbcs.xyz')

In [ ]:
groupset_i.calc_cent_mass()
groupset_i.calc_radius()
# groupset_i.calc_dl()

Create a neighbor list of groups 

In [ ]:
groupset_i.group_nblist.radii_nblist(groupset_i.strucC.lat,groupset_i.cent_mass,groupset_i.radius,radii_buffer=1.00)

In [ ]:
for gk_i,g_i in groupset_i.groups.iteritems():
        print g_i, groupset_i.group_nblist.calc_nnab(g_i)



Loop over each group, shift the group to the center of the simulation cell and write an .xyz file that includes the neighbors of the group.

In [ ]:
for gk_i,g_i in groupset_i.groups.iteritems():
        list_i = groupset_i.groups[gk_i].pkeys
        for g_j in groupset_i.group_nblist.getnbs(gk_i):
            list_i += groupset_i.groups[g_j].pkeys
        groupset_i.strucC.shift_pos(-1.0*g_i.cent_mass)  # Place center of mass at origin
        groupset_i.strucC.write_xyz_list(list_i,xyz_file='nn_{}.xyz'.format(g_i.tag))
        groupset_i.strucC.shift_pos(g_i.cent_mass)  # Return center of mass 

The nearest neighbor cluster look good so let us calculate the electron transfer 

In [ ]:
for gk_i,g_i in groupset_i.groups.iteritems():
    for g_j in groupset_i.group_nblist.getnbs(gk_i):
        if( g_j > g_i ):
            print g_i,g_j
            

Cool looks like we got the box size down a bit, but we will need to run some more to get a solid system

Let's change some of our run parameters for this longer run

In [ ]:
print peregrine.properties.keys()

In [ ]:
peregrine.properties['queue'] = 'batch'
peregrine.properties['nodes'] =  8
peregrine.properties['ppn'] =  24
peregrine.properties['nproc'] =  8*24
peregrine.properties['walltime'] =  48

In [ ]:
p3ht_et = lmp_run(p3ht_et,lmp_i.strucC,oplsaa,peregrine,md_type = 'equ1')

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_equ1_p3HTx50_equ0'] 

In [ ]:
lmp_i.analysis()

In [ ]:
print len(lmp_i.properties['run_list'])

In [ ]:
lmp_i.store()

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_3']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
lmp_i.strucC.write_xyz()

Sweet as bro!